<a href="https://colab.research.google.com/github/r0cketr1kky/PPML_Face_Rec/blob/master/ResNet_VGGFacenew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-2ffysnhp
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-2ffysnhp
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=e62797921d89ec6b106bb1973082e3fe45d27f309932e653371447f5fd662ef6
  Stored in directory: /tmp/pip-ephem-wheel-cache-0la82jft/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [2]:
!pip show keras-vggface

Name: keras-vggface
Version: 0.6
Summary: VGGFace implementation with Keras framework
Home-page: https://github.com/rcmalli/keras-vggface
Author: Refik Can MALLI
Author-email: mallir@itu.edu.tr
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: h5py, keras, numpy, pyyaml, scipy, six, pillow
Required-by: 


In [3]:
import keras_vggface

Using TensorFlow backend.


In [4]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 1.2MB/s 


In [5]:
import mtcnn

In [6]:
!pip install tf-encrypted

     |████████████████████████████████| 2.7MB 307kB/s 
     |████████████████████████████████| 276kB 4.0MB/s 
     |████████████████████████████████| 110.5MB 45kB/s 
     |████████████████████████████████| 512kB 6.3MB/s 
     |████████████████████████████████| 3.8MB 6.2MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=bc39e41449d49e89a0a4e67c537d770c92dfa473bf44886bba6cf0b1edeae9ff
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=2a44e1b34556c9a2a2cb24e4e993fef501c0b4f96b70a9211a1ce246dc0eba24
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built pyyaml gast
ERROR: Operation cancelled by user


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

KeyboardInterrupt: ignored

In [2]:
import os
import pandas as pd

fpath = '/content/gdrive/My Drive/Face_images'

f = os.listdir(fpath)

In [3]:
f

['Arsene_Wenger.jpg',
 'Channing_Tatum.jpg',
 'Suzanne_Shaw.jpg',
 'Joaquin_Phoenix.jpg',
 'Alyson_Hannigan.jpg',
 'David_Alaba.jpg',
 'Brie_Larson.jpg',
 'Sebastian_Vettel.jpg',
 'Elizabeth_Olsen.jpg',
 'Kajal_Aggarwal.jpg',
 'testimages']

In [4]:
f = f[:len(f)-1]
f

['Arsene_Wenger.jpg',
 'Channing_Tatum.jpg',
 'Suzanne_Shaw.jpg',
 'Joaquin_Phoenix.jpg',
 'Alyson_Hannigan.jpg',
 'David_Alaba.jpg',
 'Brie_Larson.jpg',
 'Sebastian_Vettel.jpg',
 'Elizabeth_Olsen.jpg',
 'Kajal_Aggarwal.jpg']

In [5]:
df = pd.DataFrame()

In [6]:
for i in range(len(f)):
  f[i] = '/content/gdrive/My Drive/Face_images/' + f[i]

f

['/content/gdrive/My Drive/Face_images/Arsene_Wenger.jpg',
 '/content/gdrive/My Drive/Face_images/Channing_Tatum.jpg',
 '/content/gdrive/My Drive/Face_images/Suzanne_Shaw.jpg',
 '/content/gdrive/My Drive/Face_images/Joaquin_Phoenix.jpg',
 '/content/gdrive/My Drive/Face_images/Alyson_Hannigan.jpg',
 '/content/gdrive/My Drive/Face_images/David_Alaba.jpg',
 '/content/gdrive/My Drive/Face_images/Brie_Larson.jpg',
 '/content/gdrive/My Drive/Face_images/Sebastian_Vettel.jpg',
 '/content/gdrive/My Drive/Face_images/Elizabeth_Olsen.jpg',
 '/content/gdrive/My Drive/Face_images/Kajal_Aggarwal.jpg']

In [7]:
'''VGGFace models for Keras.

# Notes:
- Resnet50 and VGG16  are modified architectures from Keras Application folder. [Keras](https://keras.io)

- Squeeze and excitation block is taken from  [Squeeze and Excitation Networks in
 Keras](https://github.com/titu1994/keras-squeeze-excite-network) and modified.



from tensorflow.keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D, \
    GlobalMaxPooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization, \
    AveragePooling2D, Reshape, Permute, multiply
from keras_applications.imagenet_utils import _obtain_input_shape
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from tensorflow.keras import backend as K
from keras_vggface import utils
from keras.engine.topology import get_source_inputs
import warnings
from tensorflow.keras.models import Model
from tensorflow.keras import layers


def VGG16(include_top=True, weights='vggface',
          input_tensor=None, input_shape=None,
          pooling=None,
          classes=2622):
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=48,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top)
    
    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='conv1_2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool1')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_1')(
        x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='conv2_2')(
        x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool2')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_1')(
        x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_2')(
        x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='conv3_3')(
        x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool3')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_1')(
        x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_2')(
        x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv4_3')(
        x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool4')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_1')(
        x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_2')(
        x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='conv5_3')(
        x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='pool5')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, name='fc6')(x)
        x = Activation('relu', name='fc6/relu')(x)
        x = Dense(4096, name='fc7')(x)
        x = Activation('relu', name='fc7/relu')(x)
        x = Dense(classes, name='fc8')(x)
        x = Activation('softmax', name='fc8/softmax')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

            # Ensure that the model takes into account
            # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
        # Create model.
    model = Model(inputs, x, name='vggface_vgg16')  # load weights
    if weights == 'vggface':
        if include_top:
            weights_path = get_file('rcmalli_vggface_tf_vgg16.h5',
                                    utils.
                                    VGG16_WEIGHTS_PATH,
                                    cache_subdir=utils.VGGFACE_DIR)
        else:
            weights_path = get_file('rcmalli_vggface_tf_notop_vgg16.h5',
                                    utils.VGG16_WEIGHTS_PATH_NO_TOP,
                                    cache_subdir=utils.VGGFACE_DIR)
        model.load_weights(weights_path, by_name=True)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if include_top:
                maxpool = model.get_layer(name='pool5')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc6')
                layer_utils.convert_dense_weights_data_format(dense, shape,
                                                              'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model


def resnet_identity_block(input_tensor, kernel_size, filters, stage, block,
                          bias=False):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv1_reduce_name = 'conv' + str(stage) + "_" + str(block) + "_1x1_reduce"
    conv1_increase_name = 'conv' + str(stage) + "_" + str(
        block) + "_1x1_increase"
    conv3_name = 'conv' + str(stage) + "_" + str(block) + "_3x3"

    x = Conv2D(filters1, (1, 1), use_bias=bias, name=conv1_reduce_name)(
        input_tensor)
    x = BatchNormalization(axis=bn_axis, name=conv1_reduce_name + "/bn")(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, use_bias=bias,
               padding='same', name=conv3_name)(x)
    x = BatchNormalization(axis=bn_axis, name=conv3_name + "/bn")(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), use_bias=bias, name=conv1_increase_name)(x)
    x = BatchNormalization(axis=bn_axis, name=conv1_increase_name + "/bn")(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def resnet_conv_block(input_tensor, kernel_size, filters, stage, block,
                      strides=(2, 2), bias=False):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv1_reduce_name = 'conv' + str(stage) + "_" + str(block) + "_1x1_reduce"
    conv1_increase_name = 'conv' + str(stage) + "_" + str(
        block) + "_1x1_increase"
    conv1_proj_name = 'conv' + str(stage) + "_" + str(block) + "_1x1_proj"
    conv3_name = 'conv' + str(stage) + "_" + str(block) + "_3x3"

    x = Conv2D(filters1, (1, 1), strides=strides, use_bias=bias,
               name=conv1_reduce_name)(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=conv1_reduce_name + "/bn")(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', use_bias=bias,
               name=conv3_name)(x)
    x = BatchNormalization(axis=bn_axis, name=conv3_name + "/bn")(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv1_increase_name, use_bias=bias)(x)
    x = BatchNormalization(axis=bn_axis, name=conv1_increase_name + "/bn")(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides, use_bias=bias,
                      name=conv1_proj_name)(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=conv1_proj_name + "/bn")(
        shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x


def RESNET50(include_top=True, weights='vggface',
             input_tensor=None, input_shape=None,
             pooling=None,
             classes=8631):
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=32,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top,
                                      weights=weights)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = Conv2D(
        64, (7, 7), use_bias=False, strides=(2, 2), padding='same',
        name='conv1/7x7_s2')(img_input)
    x = BatchNormalization(axis=bn_axis, name='conv1/7x7_s2/bn')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = resnet_conv_block(x, 3, [64, 64, 256], stage=2, block=1, strides=(1, 1))
    x = resnet_identity_block(x, 3, [64, 64, 256], stage=2, block=2)
    x = resnet_identity_block(x, 3, [64, 64, 256], stage=2, block=3)

    x = resnet_conv_block(x, 3, [128, 128, 512], stage=3, block=1)
    x = resnet_identity_block(x, 3, [128, 128, 512], stage=3, block=2)
    x = resnet_identity_block(x, 3, [128, 128, 512], stage=3, block=3)
    x = resnet_identity_block(x, 3, [128, 128, 512], stage=3, block=4)

    x = resnet_conv_block(x, 3, [256, 256, 1024], stage=4, block=1)
    x = resnet_identity_block(x, 3, [256, 256, 1024], stage=4, block=2)
    x = resnet_identity_block(x, 3, [256, 256, 1024], stage=4, block=3)
    x = resnet_identity_block(x, 3, [256, 256, 1024], stage=4, block=4)
    x = resnet_identity_block(x, 3, [256, 256, 1024], stage=4, block=5)
    x = resnet_identity_block(x, 3, [256, 256, 1024], stage=4, block=6)

    x = resnet_conv_block(x, 3, [512, 512, 2048], stage=5, block=1)
    x = resnet_identity_block(x, 3, [512, 512, 2048], stage=5, block=2)
    x = resnet_identity_block(x, 3, [512, 512, 2048], stage=5, block=3)

    x = AveragePooling2D((7, 7), name='avg_pool')(x)

    if include_top:
        x = Flatten()(x)
        x = Dense(classes, activation='softmax', name='classifier')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vggface_resnet50')

    # load weights
    if weights == 'vggface':
        if include_top:
            weights_path = get_file('rcmalli_vggface_tf_resnet50.h5',
                                    utils.RESNET50_WEIGHTS_PATH,
                                    cache_subdir=utils.VGGFACE_DIR)
        else:
            weights_path = get_file('rcmalli_vggface_tf_notop_resnet50.h5',
                                    utils.RESNET50_WEIGHTS_PATH_NO_TOP,
                                    cache_subdir=utils.VGGFACE_DIR)
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)
            if include_top:
                maxpool = model.get_layer(name='avg_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='classifier')
                layer_utils.convert_dense_weights_data_format(dense, shape,
                                                              'channels_first')

        if K.image_data_format() == 'channels_first' and K.backend() == 'tensorflow':
            warnings.warn('You are using the TensorFlow backend, yet you '
                          'are using the Theano '
                          'image data format convention '
                          '(`image_data_format="channels_first"`). '
                          'For best performance, set '
                          '`image_data_format="channels_last"` in '
                          'your Keras config '
                          'at ~/.keras/keras.json.')
    elif weights is not None:
        model.load_weights(weights)

    return model


def senet_se_block(input_tensor, stage, block, compress_rate=16, bias=False):
    conv1_down_name = 'conv' + str(stage) + "_" + str(
        block) + "_1x1_down"
    conv1_up_name = 'conv' + str(stage) + "_" + str(
        block) + "_1x1_up"

    num_channels = int(input_tensor.shape[-1])
    bottle_neck = int(num_channels // compress_rate)

    se = GlobalAveragePooling2D()(input_tensor)
    se = Reshape((1, 1, num_channels))(se)
    se = Conv2D(bottle_neck, (1, 1), use_bias=bias,
                name=conv1_down_name)(se)
    se = Activation('relu')(se)
    se = Conv2D(num_channels, (1, 1), use_bias=bias,
                name=conv1_up_name)(se)
    se = Activation('sigmoid')(se)

    x = input_tensor
    x = multiply([x, se])
    return x


def senet_conv_block(input_tensor, kernel_size, filters,
                     stage, block, bias=False, strides=(2, 2)):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    bn_eps = 0.0001

    conv1_reduce_name = 'conv' + str(stage) + "_" + str(block) + "_1x1_reduce"
    conv1_increase_name = 'conv' + str(stage) + "_" + str(
        block) + "_1x1_increase"
    conv1_proj_name = 'conv' + str(stage) + "_" + str(block) + "_1x1_proj"
    conv3_name = 'conv' + str(stage) + "_" + str(block) + "_3x3"

    x = Conv2D(filters1, (1, 1), use_bias=bias, strides=strides,
               name=conv1_reduce_name)(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=conv1_reduce_name + "/bn",epsilon=bn_eps)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', use_bias=bias,
               name=conv3_name)(x)
    x = BatchNormalization(axis=bn_axis, name=conv3_name + "/bn",epsilon=bn_eps)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv1_increase_name, use_bias=bias)(x)
    x = BatchNormalization(axis=bn_axis, name=conv1_increase_name + "/bn" ,epsilon=bn_eps)(x)

    se = senet_se_block(x, stage=stage, block=block, bias=True)

    shortcut = Conv2D(filters3, (1, 1), use_bias=bias, strides=strides,
                      name=conv1_proj_name)(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis,
                                  name=conv1_proj_name + "/bn",epsilon=bn_eps)(shortcut)

    m = layers.add([se, shortcut])
    m = Activation('relu')(m)
    return m


def senet_identity_block(input_tensor, kernel_size,
                         filters, stage, block, bias=False):
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    bn_eps = 0.0001

    conv1_reduce_name = 'conv' + str(stage) + "_" + str(block) + "_1x1_reduce"
    conv1_increase_name = 'conv' + str(stage) + "_" + str(
        block) + "_1x1_increase"
    conv3_name = 'conv' + str(stage) + "_" + str(block) + "_3x3"

    x = Conv2D(filters1, (1, 1), use_bias=bias,
               name=conv1_reduce_name)(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=conv1_reduce_name + "/bn",epsilon=bn_eps)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', use_bias=bias,
               name=conv3_name)(x)
    x = BatchNormalization(axis=bn_axis, name=conv3_name + "/bn",epsilon=bn_eps)(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv1_increase_name, use_bias=bias)(x)
    x = BatchNormalization(axis=bn_axis, name=conv1_increase_name + "/bn",epsilon=bn_eps)(x)

    se = senet_se_block(x, stage=stage, block=block, bias=True)

    m = layers.add([se, input_tensor])
    m = Activation('relu')(m)

    return m


def SENET50(include_top=True, weights='vggface',
            input_tensor=None, input_shape=None,
            pooling=None,
            classes=8631):
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=197,
                                      data_format=K.image_data_format(),
                                      require_flatten=include_top,
                                      weights=weights)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    bn_eps = 0.0001

    x = Conv2D(
        64, (7, 7), use_bias=False, strides=(2, 2), padding='same',
        name='conv1/7x7_s2')(img_input)
    x = BatchNormalization(axis=bn_axis, name='conv1/7x7_s2/bn',epsilon=bn_eps)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = senet_conv_block(x, 3, [64, 64, 256], stage=2, block=1, strides=(1, 1))
    x = senet_identity_block(x, 3, [64, 64, 256], stage=2, block=2)
    x = senet_identity_block(x, 3, [64, 64, 256], stage=2, block=3)

    x = senet_conv_block(x, 3, [128, 128, 512], stage=3, block=1)
    x = senet_identity_block(x, 3, [128, 128, 512], stage=3, block=2)
    x = senet_identity_block(x, 3, [128, 128, 512], stage=3, block=3)
    x = senet_identity_block(x, 3, [128, 128, 512], stage=3, block=4)

    x = senet_conv_block(x, 3, [256, 256, 1024], stage=4, block=1)
    x = senet_identity_block(x, 3, [256, 256, 1024], stage=4, block=2)
    x = senet_identity_block(x, 3, [256, 256, 1024], stage=4, block=3)
    x = senet_identity_block(x, 3, [256, 256, 1024], stage=4, block=4)
    x = senet_identity_block(x, 3, [256, 256, 1024], stage=4, block=5)
    x = senet_identity_block(x, 3, [256, 256, 1024], stage=4, block=6)

    x = senet_conv_block(x, 3, [512, 512, 2048], stage=5, block=1)
    x = senet_identity_block(x, 3, [512, 512, 2048], stage=5, block=2)
    x = senet_identity_block(x, 3, [512, 512, 2048], stage=5, block=3)

    x = AveragePooling2D((7, 7), name='avg_pool')(x)

    if include_top:
        x = Flatten()(x)
        x = Dense(classes, activation='softmax', name='classifier')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='vggface_senet50')

    # load weights
    if weights == 'vggface':
        if include_top:
            weights_path = get_file('rcmalli_vggface_tf_senet50.h5',
                                    utils.SENET50_WEIGHTS_PATH,
                                    cache_subdir=utils.VGGFACE_DIR)
        else:
            weights_path = get_file('rcmalli_vggface_tf_notop_senet50.h5',
                                    utils.SENET50_WEIGHTS_PATH_NO_TOP,
                                    cache_subdir=utils.VGGFACE_DIR)
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)
            if include_top:
                maxpool = model.get_layer(name='avg_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='classifier')
                layer_utils.convert_dense_weights_data_format(dense, shape,
                                                              'channels_first')

        if K.image_data_format() == 'channels_first' and K.backend() == 'tensorflow':
            warnings.warn('You are using the TensorFlow backend, yet you '
                          'are using the Theano '
                          'image data format convention '
                          '(`image_data_format="channels_first"`). '
                          'For best performance, set '
                          '`image_data_format="channels_last"` in '
                          'your Keras config '
                          'at ~/.keras/keras.json.')
    elif weights is not None:
        model.load_weights(weights)

    return model

'''

'VGGFace models for Keras.\n\n# Notes:\n- Resnet50 and VGG16  are modified architectures from Keras Application folder. [Keras](https://keras.io)\n\n- Squeeze and excitation block is taken from  [Squeeze and Excitation Networks in\n Keras](https://github.com/titu1994/keras-squeeze-excite-network) and modified.\n\n\n\nfrom tensorflow.keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D,     GlobalMaxPooling2D, Activation, Conv2D, MaxPooling2D, BatchNormalization,     AveragePooling2D, Reshape, Permute, multiply\nfrom keras_applications.imagenet_utils import _obtain_input_shape\nfrom keras.utils import layer_utils\nfrom keras.utils.data_utils import get_file\nfrom tensorflow.keras import backend as K\nfrom keras_vggface import utils\nfrom keras.engine.topology import get_source_inputs\nimport warnings\nfrom tensorflow.keras.models import Model\nfrom tensorflow.keras import layers\n\n\ndef VGG16(include_top=True, weights=\'vggface\',\n          input_tensor=None, input_shape=

In [1]:
'''VGGFace models for Keras.

# Reference:
- [Deep Face Recognition](http://www.robots.ox.ac.uk/~vgg/publications/2015/Parkhi15/parkhi15.pdf)
- [VGGFace2: A dataset for recognising faces across pose and age](http://www.robots.ox.ac.uk/~vgg/data/vgg_face2/vggface2.pdf)


from __future__ import print_function


def VGGFace(include_top=True, model='vgg16', weights='vggface',
            input_tensor=None, input_shape=None,
            pooling=None,
            classes=None):
    """Instantiates the VGGFace architectures.
    Optionally loads weights pre-trained
    on VGGFace datasets. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization)
            or "vggface" (pre-training on VGGFACE datasets).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        model: selects the one of the available architectures 
            vgg16, resnet50 or senet50 default is vgg16.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 48.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """

    if weights not in {'vggface', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `vggface`'
                         '(pre-training on VGGFace Datasets).')

    if model == 'vgg16':

        if classes is None:
            classes = 2622

        if weights == 'vggface' and include_top and classes != 2622:
            raise ValueError(
                'If using `weights` as vggface original with `include_top`'
                ' as true, `classes` should be 2622')

        return VGG16(include_top=include_top, input_tensor=input_tensor,
                     input_shape=input_shape, pooling=pooling,
                     weights=weights,
                     classes=classes)


    if model == 'resnet50':

        if classes is None:
            classes = 8631

        if weights == 'vggface' and include_top and classes != 8631:
            raise ValueError(
                'If using `weights` as vggface original with `include_top`'
                ' as true, `classes` should be 8631')
            

        return RESNET50(include_top=include_top, input_tensor=input_tensor,
                        input_shape=input_shape, pooling=pooling,
                        weights=weights,
                        classes=classes)

    if model == 'senet50':

        if classes is None:
            classes = 8631

        if weights == 'vggface' and include_top and classes != 8631:
            raise ValueError(
                'If using `weights` as vggface original with `include_top`'
                ' as true, `classes` should be 8631')

        return SENET50(include_top=include_top, input_tensor=input_tensor,
                        input_shape=input_shape, pooling=pooling,
                        weights=weights,
                        classes=classes)
'''

SyntaxError: ignored

In [9]:
_# face verification with the VGGFace2 model
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]
	# convert into an array of samples
	samples = asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# create a vggface model
  print(samples)
  print(samples.shape)
	model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
	# perform prediction
	yhat = model.predict(samples)
	return yhat

embeddings = get_embeddings(f)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.





In [10]:
embeddings

array([[2.6663566e-02, 9.6299648e-01, 1.7681995e-01, ..., 1.1269286e+00,
        1.8694632e-01, 0.0000000e+00],
       [3.4382320e-03, 3.2605079e-01, 8.8442338e-01, ..., 1.0139080e-01,
        8.1121981e-01, 4.4080889e-01],
       [7.2877874e+00, 3.1905100e-01, 1.2724630e+00, ..., 3.5127757e+00,
        8.7759703e-01, 1.5965286e-04],
       ...,
       [1.3959834e+01, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        2.3562360e+00, 1.0428590e+00],
       [1.2057389e+00, 1.1040424e+00, 3.2777426e+00, ..., 0.0000000e+00,
        0.0000000e+00, 2.2961733e+00],
       [1.9995850e-02, 0.0000000e+00, 1.3471788e+00, ..., 3.7983835e+00,
        1.9359640e+00, 2.6511335e+00]], dtype=float32)

In [11]:
len(embeddings)

10

In [12]:
for i in range(len(f)):
  f[i] = f[i].split('/')[5]

f

['Arsene_Wenger.jpg',
 'Channing_Tatum.jpg',
 'Suzanne_Shaw.jpg',
 'Joaquin_Phoenix.jpg',
 'Alyson_Hannigan.jpg',
 'David_Alaba.jpg',
 'Brie_Larson.jpg',
 'Sebastian_Vettel.jpg',
 'Elizabeth_Olsen.jpg',
 'Kajal_Aggarwal.jpg']

In [13]:
for i in range(len(f)-1):
  df[f[i]] = list(embeddings[i])

In [14]:
df

,Arsene_Wenger.jpg,Channing_Tatum.jpg,Suzanne_Shaw.jpg,Joaquin_Phoenix.jpg,Alyson_Hannigan.jpg,David_Alaba.jpg,Brie_Larson.jpg,Sebastian_Vettel.jpg,Elizabeth_Olsen.jpg
0,0.026664,0.003438,7.287787,2.615009,10.211094,5.204028,0.792291,13.959834,1.205739
1,0.962996,0.326051,0.319051,0.073866,0.000000,0.795300,0.660623,0.000000,1.104042
2,0.176820,0.884423,1.272463,0.022776,0.000000,0.422715,0.652062,0.000000,3.277743
3,0.025856,0.799032,0.000000,0.011135,3.377105,1.023647,0.000000,0.735233,0.448290
4,0.000000,5.094138,3.480408,1.709928,0.099040,0.322407,4.749907,0.000000,0.643640
...,...,...,...,...,...,...,...,...,...
2043,0.017205,0.000000,0.006769,3.279908,1.200256,0.542951,0.010526,0.072805,0.000000
2044,0.557395,0.486283,0.000000,0.000000,0.174508,0.000000,0.245049,1.620390,0.044568
2045,1.126929,0.101391,3.512776,0.000000,0.016456,5.398932,9.186919,0.000000,0.000000
2046,0.186946,0.811220,0.877597,4.503322,0.000000,0.783289,2.525741,2.356236,0.000000


In [15]:
df[f[8]]

0       1.205739
1       1.104042
2       3.277743
3       0.448290
4       0.643640
          ...   
2043    0.000000
2044    0.044568
2045    0.000000
2046    0.000000
2047    2.296173
Name: Elizabeth_Olsen.jpg, Length: 2048, dtype: float64

In [16]:
embeddings[8]

array([1.2057389, 1.1040424, 3.2777426, ..., 0.       , 0.       ,
       2.2961733], dtype=float32)

In [17]:
new_embed = get_embeddings([fpath+'/testimages/eo1.jpg'])

In [18]:
new_embed

array([[8.322706  , 4.7730403 , 2.5647454 , ..., 0.17429346, 1.0706133 ,
        0.        ]], dtype=float32)

In [19]:
ne = new_embed

In [20]:
emb = embeddings

In [21]:
import numpy as np

In [22]:
import tensorflow as tf

In [23]:
emb1 = []
for i in range(len(emb)):
  data_np = np.asarray(emb[i], np.float32)
  data_em = tf.convert_to_tensor(data_np, np.float32)
  emb1.append(data_em)

In [24]:
lamb1 = np.array(0.5)
data_np = np.asarray(lamb1, np.float32)
data_lamb1 = tf.convert_to_tensor(data_np, np.float32)

In [25]:
data_np = np.asarray(ne, np.float32)
data_em = tf.convert_to_tensor(data_np, np.float32)

In [35]:
import tensorflow as tf
import numpy as np
import tf_encrypted as tfe

'''
@tfe.local_computation('players')
def provide_input():
    # normal TensorFlow operations can be run locally
    # as part of defining a private input, in this
    # case on the machine of the input provider
    
    newl = emb

    
    return newl
'''


prot = tfe.protocol.SecureNN()
tfe.set_protocol(prot)

  # define inputs
w = tfe.define_private_variable(data_em)

x = []
for i in range(len(emb1)):
  x.append(tfe.define_private_variable(emb1[i]))


print(w)
print(x)

y = []
for i in range(len(x)):
  y.append(tfe.sub(x[i], w))


lessthan = []
lambdaa = tfe.define_private_variable(data_lamb1)
for i in range(len(x)):
  ynew = tfe.less(y[i], lambdaa)
  lessthan.append(ynew)

print(lessthan)
'''
def lessthan(d, lam=0.5):
    if d < lam:
      return 1
    else:
      return 0

  newl = []
  for i in range(len(y)):
    newl.append(lessthan(y[i]))

'''

with tfe.Session() as sess:
  # initialize variables
  sess.run(tfe.global_variables_initializer())
  # reveal result
  res = []
  for i in range(len(lessthan)):
    result = sess.run(lessthan[i].reveal())
    res.append(result)


PondPrivateVariable(shape=(1, 2048))
[PondPrivateVariable(shape=(2048,)), PondPrivateVariable(shape=(2048,)), PondPrivateVariable(shape=(2048,)), PondPrivateVariable(shape=(2048,)), PondPrivateVariable(shape=(2048,)), PondPrivateVariable(shape=(2048,)), PondPrivateVariable(shape=(2048,)), PondPrivateVariable(shape=(2048,)), PondPrivateVariable(shape=(2048,)), PondPrivateVariable(shape=(2048,))]
[PondPrivateTensor(shape=(1, 2048)), PondPrivateTensor(shape=(1, 2048)), PondPrivateTensor(shape=(1, 2048)), PondPrivateTensor(shape=(1, 2048)), PondPrivateTensor(shape=(1, 2048)), PondPrivateTensor(shape=(1, 2048)), PondPrivateTensor(shape=(1, 2048)), PondPrivateTensor(shape=(1, 2048)), PondPrivateTensor(shape=(1, 2048)), PondPrivateTensor(shape=(1, 2048))]


In [36]:
print(res)

[array([[1, 1, 1, ..., 0, 1, 1]]), array([[1, 1, 1, ..., 1, 1, 1]]), array([[1, 1, 1, ..., 0, 1, 1]]), array([[1, 1, 1, ..., 1, 0, 1]]), array([[0, 1, 1, ..., 1, 1, 0]]), array([[1, 1, 1, ..., 0, 1, 0]]), array([[1, 1, 1, ..., 0, 0, 1]]), array([[0, 1, 1, ..., 1, 0, 0]]), array([[1, 1, 0, ..., 1, 1, 0]]), array([[1, 1, 1, ..., 0, 0, 0]])]


In [39]:
for i in range(len(res)):
  num_ones = (res[i] == 1).sum()
  print(num_ones)

1371
1441
1460
1444
1472
1457
1412
1377
1480
1425


In [ ]:
# Example of face detection with a vggface2 model
from numpy import expand_dims
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# load the photo and extract the face
pixels = extract_face('/content/gdrive/My Drive/Face_images/Channing_Tatum.jpg')
# convert one face into samples
pixels = pixels.astype('float32')
samples = expand_dims(pixels, axis=0)
# prepare the face for the model, e.g. center pixels
samples = preprocess_input(samples, version=2)
# create a vggface model
model = VGGFace(model='resnet50')
# perform prediction
yhat = model.predict(samples)
print(yhat)
print(len(yhat[0]))
# convert prediction into names
results = decode_predictions(yhat)
# display most likely results
for result in results[0]:
	print('%s: %.3f%%' % (result[0], result[1]*100))

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



[[1.0087899e-07 5.1767080e-07 2.1022200e-07 ... 3.6320075e-06
  1.4810366e-06 7.8079688e-07]]
8631
b' Channing_Tatum': 91.401%
b' Jeff_Corwin': 0.408%
b' Jill_Goodacre': 0.251%
b' Vince_Gill': 0.227%
b' Ryan_Peake': 0.204%


In [ ]:
# face verification with the VGGFace2 model
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	# detect faces in the image
	results = detector.detect_faces(pixels)
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array

# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]
	# convert into an array of samples
	samples = asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# create a vggface model
	model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
	# perform prediction
	yhat = model.predict(samples)
	return yhat

# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
	else:
		print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))

# define filenames
filenames = ['/content/gdrive/My Drive/Face_images/Channing_Tatum.jpg', \
             '/content/gdrive/My Drive/Face_images/Barack_Obama.jpg', \
             '/content/gdrive/My Drive/Face_images/Arsene_Wenger.jpg', \
             '/content/gdrive/My Drive/channing_tatum.jpg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
# define sharon stone
sharon_id = embeddings[0]
# verify known photos of sharon
print('Positive Tests')
is_match(embeddings[0], embeddings[1])
is_match(embeddings[0], embeddings[2])
# verify known photos of other people
print('Negative Tests')
is_match(embeddings[0], embeddings[3])

Positive Tests
>face is NOT a Match (0.782 > 0.500)
>face is NOT a Match (0.770 > 0.500)
Negative Tests
>face is a Match (0.300 <= 0.500)


In [ ]:
sharon_id

array([0.00343823, 0.3260508 , 0.8844234 , ..., 0.1013908 , 0.8112198 ,
       0.4408089 ], dtype=float32)

In [ ]:
len(sharon_id)

2048

In [ ]:
import tensorflow as tf
import tf_encrypted as tfe

@tfe.local_computation('input-provider')
def provide_input():
    # normal TensorFlow operations can be run locally
    # as part of defining a private input, in this
    # case on the machine of the input provider
    return tf.ones(shape=(5, 10))

# define inputs
w = tfe.define_private_variable(tf.ones(shape=(10,10)))
print(w)
x = provide_input()
print(x)

# define computation
y = tfe.matmul(x, w)
print(y)

with tfe.Session() as sess:
    # initialize variables
    sess.run(tfe.global_variables_initializer())
    # reveal result
    result = sess.run(y.reveal())

PondPrivateVariable(shape=(10, 10))
PondPrivateTensor(shape=(5, 10))
PondPrivateTensor(shape=(5, 10))


In [ ]:
print(result)

[[10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
 [10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]]


In [ ]:
import tensorflow as tf
import numpy as np

import tf_encrypted as tfe
from tf_encrypted.protocol.aby3 import ABY3
from tf_encrypted.protocol.aby3 import ARITHMETIC
from tf_encrypted.protocol.aby3 import BOOLEAN

tf.reset_default_graph()

prot = ABY3()
tfe.set_protocol(prot)

x = tfe.define_private_variable(tf.ones(shape=(2, 2)))
y = tfe.define_private_variable(tf.ones(shape=(2, 2)))

print(x)
print(y)


z = x + y
print(z)

with tfe.Session() as sess:
    # initialize variables
    sess.run(tfe.global_variables_initializer())
    # reveal result
    result = sess.run(z.reveal())
    print(result)

ModuleNotFoundError: ignored